In [1]:
from tp5 import *
from pathlib import Path
%load_ext tensorboard

#Taille du batch
BATCH_SIZE = 32
PATH = "../data/"
MAX_LEN = 100
VOCAB_SIZE = len(lettre2id)

data_trump = DataLoader(TrumpDataset(open(PATH+"trump_full_speech.txt","rb").read().decode(),maxlen=MAX_LEN),collate_fn=pad_collate_fn, batch_size= BATCH_SIZE, shuffle=True)

In [2]:
VECT_EMB_SIZE = 100
embedding = torch.nn.Embedding(num_embeddings=VOCAB_SIZE, embedding_dim=VECT_EMB_SIZE)

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [4]:
class State :
    def __init__ (self, model, optim, emb):
        self.model = model
        self.optim = optim
        self.emb = emb
        self.epoch, self.iteration = 0, 0

## RNN

In [5]:
savepath = Path("RNN.plt")
HIDDEN = 60
lr = 0.001
NB_EPOCH = 50
if savepath.is_file():
    with savepath.open("rb") as fp :
        state = torch.load(fp)
else:
    rnn_model = RNN(in_dim=VECT_EMB_SIZE, out_dim=VOCAB_SIZE, hidden=HIDDEN).to(device)
    optim = torch.optim.Adam(params=rnn_model.parameters(),lr=lr)
    state = State(rnn_model, optim, embedding)

loss = maskedCrossEntropy

In [6]:
writer = SummaryWriter()
for epoch in range(state.epoch, NB_EPOCH):
    print(f'Epoch {epoch+1}')
    for x, y in data_trump:
        x = embedding(x)
        bz = x.shape[1]
        h = state.model.init_h0(bz)
        h = state.model.forward(x, h)
        ypred = state.model.decode(h)

        state.optim.zero_grad()
        l = loss(ypred, y, padcar=PAD_IX)
        l.backward()
        state.optim.step()
        state.iteration += 1
        writer.add_scalar('Loss/train', l, state.iteration)
        
    with savepath.open("wb") as fp :
        state.epoch = epoch + 1
        torch.save(state, fp)
writer.close()

Epoch 1


/Users/son/Documents/M2/AMAL/tme5/src/textloader.py:56: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  t = torch.tensor(t, dtype=torch.long)


Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29
Epoch 30
Epoch 31
Epoch 32
Epoch 33
Epoch 34
Epoch 35
Epoch 36
Epoch 37
Epoch 38
Epoch 39
Epoch 40
Epoch 41
Epoch 42
Epoch 43
Epoch 44
Epoch 45
Epoch 46
Epoch 47
Epoch 48
Epoch 49
Epoch 50


In [6]:
text = generate(state.model, state.emb, None, EOS_IX, "T", 200)
code2string(text)

'The cirizanss.ained 47 $5,800 the United elengele noth overs.e accagec a likence America, vioded $150 trated Relocy come a Trump American.on relounte the simple, ligs birwing firstage are no 108 Pation'

In [7]:
text = generate_beam(state.model, state.emb, None, 3, EOS_IX, "T", 200)
code2string(text)

'The was the was the was the was the was the was the was the was the was the was the was the was the was the was the was the was the was the was the was the was the was the was the was the was the was t'

In [8]:
text = generate_p_nucleus(state.model, state.emb, None, EOS_IX, "T", 200,alpha=0.95)
code2string(text)

"Town Hillary Clinton Classiom, lapaster, our in offeme takes.ans.e. and, we'll String country denels of conninn.o Organown interalk and Mexting he piate Chiablis doing to others paigs, a maching of Mut"

## LSTM

In [9]:
savepath = Path("LSTM.plt")
HIDDEN = 60
lr = 0.001
NB_EPOCH = 50
if savepath.is_file():
    with savepath.open("rb") as fp :
        state = torch.load(fp)
else:
    lstm_model = LSTM(input_size=VECT_EMB_SIZE, hidden_size=HIDDEN, vocab_size=VOCAB_SIZE).to(device)
    optim = torch.optim.Adam(params=lstm_model.parameters(),lr=lr)
    state = State(lstm_model, optim, embedding)

loss = maskedCrossEntropy

In [9]:
writer = SummaryWriter()
for epoch in range(state.epoch, NB_EPOCH):
    print(f'Epoch {epoch+1}')
    for x, y in data_trump:
        x = embedding(x)
        bz = x.shape[1]
        c, h = state.model.init_long_short_term(bz)
        _, h = state.model.forward(x, c, h)
        ypred = state.model.decode(h)

        state.optim.zero_grad()
        l = loss(ypred, y, padcar=PAD_IX)
        l.backward()
        state.optim.step()
        state.iteration += 1
        writer.add_scalar('Loss/train', l, state.iteration)
        
    with savepath.open("wb") as fp :
        state.epoch = epoch + 1
        torch.save(state, fp)
writer.close()

Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29
Epoch 30
Epoch 31
Epoch 32
Epoch 33
Epoch 34
Epoch 35
Epoch 36
Epoch 37
Epoch 38
Epoch 39
Epoch 40
Epoch 41
Epoch 42
Epoch 43
Epoch 44
Epoch 45
Epoch 46
Epoch 47
Epoch 48
Epoch 49
Epoch 50


In [10]:
text = generate(state.model, state.emb, None, EOS_IX, "T", 200)
code2string(text)

'Thank you. thnes, private citizis in GE, he was this us colory. olde addden smallitity dollass.ate. problems.in. promised from 8 your violently I said thrin. ffell.readly in manoris of the inner warldr'

In [11]:
text = generate_beam(state.model, state.emb, None, 3, EOS_IX, "T", 200)
code2string(text)

'The people of the state the state the state the state the state the state the state the state the state the state the state the state the state the state the state the state the state the state the sta'

In [12]:
text = generate_p_nucleus(state.model, state.emb, None, EOS_IX, "T", 200,alpha=0.95)
code2string(text)

'T.um," while to grap. Postices borday. to governments agence immigration.ed dolsts we mean on this in for strong, a lot your countles come of and addderia. getter supposilition on country I am vies, yo'

## GRU

In [13]:
savepath = Path("GRU.plt")
HIDDEN = 60
lr = 0.001
NB_EPOCH = 50
if savepath.is_file():
    with savepath.open("rb") as fp :
        state = torch.load(fp)
else:
    gru_model = GRU(input_size=VECT_EMB_SIZE, hidden_size=HIDDEN, vocab_size=VOCAB_SIZE).to(device)
    optim = torch.optim.Adam(params=gru_model.parameters(),lr=lr)
    state = State(gru_model, optim, embedding)

loss = maskedCrossEntropy

In [6]:
writer = SummaryWriter()
for epoch in range(state.epoch, NB_EPOCH):
    print(f'Epoch {epoch+1}')
    for x, y in data_trump:
        x = embedding(x)
        bz = x.shape[1]
        h = state.model.init_memory(bz)
        h = state.model.forward(x, h)
        ypred = state.model.decode(h)

        state.optim.zero_grad()
        l = loss(ypred, y, padcar=PAD_IX)
        l.backward()
        state.optim.step()
        state.iteration += 1
        writer.add_scalar('Loss/train', l, state.iteration)
        
    with savepath.open("wb") as fp :
        state.epoch = epoch + 1
        torch.save(state, fp)
writer.close()

Epoch 1


/Users/son/Documents/M2/AMAL/tme5/src/textloader.py:56: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  t = torch.tensor(t, dtype=torch.long)


Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29
Epoch 30
Epoch 31
Epoch 32
Epoch 33
Epoch 34
Epoch 35
Epoch 36
Epoch 37
Epoch 38
Epoch 39
Epoch 40
Epoch 41
Epoch 42
Epoch 43
Epoch 44
Epoch 45
Epoch 46
Epoch 47
Epoch 48
Epoch 49
Epoch 50


In [14]:
text = generate(state.model, state.emb, None, EOS_IX, "T", 200)
code2string(text)

'The well. you will gloo. she do wattht indicenty incredece of Department rene disastever of wars. builds busine statianal comptith 2.By Afrill. very establyon, Ware] American schools open new means to '

In [15]:
text = generate_beam(state.model, state.emb, None, EOS_IX, k=3, start="B", maxlen=150)
code2string(text)

'Buteregedduthithallintatthionerkinirelys origenineraly temarordoly ac.rdreatharal arouchino.aneaconesechedinor tapousticaledofouenitevaned. athalinofor'

In [16]:
text = generate_p_nucleus(state.model, state.emb, None, EOS_IX, "T", 200,alpha=0.95)
code2string(text)

'This can\'t vote. signing most how whorrord and protect from the president reaching. destructurs. handious suring economic one in this proyearly and badges smarding. befunding 7 powed "enside. this poli'

In [13]:
%tensorboard --logdir runs

Reusing TensorBoard on port 6006 (pid 3523), started 0:00:02 ago. (Use '!kill 3523' to kill it.)